### K-Nearest Neighbor for Genre Classification
In this exercise, we experiment with a k-nearest neighbor learner and evaluate it using 10-fold cross validation. We leverage the Scikit Learn library to build a classifier and separate our data into folds for cross validation.

After receiving our mean accuracy, we compare that accuracy with a baseline classification algorithm's (ZeroR) expected accuracy.

In [73]:
import pandas as pd
import autograd.numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier

In [78]:
# A helper function to extract attribute columns from our dataset
def include_only(x, attribute_names, attribute_subset):
    excluded_columns = [i for i, attribute in enumerate(attribute_names) if attribute not in attribute_subset]
    return np.delete(x, excluded_columns, axis=1)

In this section, we attempt to train a K-Nearest Neighbor Classifier with a select subset of attributes, shown below.

In [80]:
# Read data
DATA_BASE_URL = "https://raw.githubusercontent.com/sql-injection/spotify_data/master/"
datasets = {
    "train": DATA_BASE_URL + "train.csv",
    "test": DATA_BASE_URL + "test.csv",
    "all": DATA_BASE_URL + "spotify.csv"
}

total_df = pd.read_csv(datasets["all"])
attribute_names = list(total_df)[:-1]
x = total_df[attribute_names].values
y = total_df["Class"].values[:, np.newaxis]

attribute_subset = ["danceability", "energy", "speechiness", "valence"]
print("Attributes we are considering:", attribute_subset)
original_x = np.copy(x)
original_y = np.copy(y)
x = include_only(original_x, attribute_names, attribute_subset)

Attributes we are considering: ['danceability', 'energy', 'speechiness', 'valence']


In [81]:
class ValidationResult(object):
    def __init__(self):
        self.accuracy = 0.0
        self.results = list()
        
    def __str__(self):
        errors = [(actual, predicted) for actual, predicted in self.results if actual != predicted]
        error_length = len(errors)
        total_length = len(self.results)
        return "<accuracy={accuracy}, num_incorrect={incorrect}, num_correct={correct}, errors={errors}>\n".format(
            accuracy=self.accuracy,
            incorrect=error_length,
            correct=total_length - error_length,
            errors=errors
        )


def accuracy(y_pred, y_test):
    correct = 0
    total = len(y_test)
    v = ValidationResult()
    for i in range(total):
        v.results.append((y_test[i][0], y_pred[i]))
        if y_test[i][0] == y_pred[i]:
            correct += 1
    v.accuracy = correct / total 
    return v

We find the optimal k for our knn classifier to be 2 neighbors. We then perform 10-fold cross validation on our dataset, evaluate the accuracy of each fold, and take the mean accuracy of all folds.

In [71]:
num_splits = 10
num_neighbors = 3
kf = KFold(n_splits=num_splits)
cross_validations = list()

for train_index, test_index in kf.split(x):
    # Split into testing and training
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit
    classifier = KNeighborsClassifier(n_neighbors=num_neighbors)
    classifier.fit(x_train, y_train.ravel())
    
    # Evaluate
    y_pred = classifier.predict(x_test)
    cross_validations.append(accuracy(y_pred, y_test))

for i, v in enumerate(cross_validations):
    print("Split", i + 1)
    print(v)
    
print("Mean accuracy:", np.mean([v.accuracy for v in cross_validations]))

Split 1
<accuracy=0.07317073170731707, num_incorrect=38, num_correct=3, errors=[('edm', 'country'), ('edm', 'country'), ('edm', 'country'), ('edm', 'country'), ('edm', 'country'), ('edm', 'country'), ('edm', 'rock'), ('edm', 'country'), ('edm', 'pop'), ('edm', 'country'), ('edm', 'country'), ('edm', 'country'), ('edm', 'rock'), ('edm', 'country'), ('edm', 'pop'), ('edm', 'pop'), ('edm', 'country'), ('edm', 'pop'), ('edm', 'rock'), ('edm', 'rock'), ('edm', 'rock'), ('edm', 'country'), ('edm', 'country'), ('edm', 'country'), ('edm', 'country'), ('edm', 'rock'), ('edm', 'country'), ('edm', 'country'), ('edm', 'country'), ('edm', 'country'), ('edm', 'rock'), ('edm', 'country'), ('edm', 'rock'), ('edm', 'rock'), ('edm', 'rock'), ('edm', 'rock'), ('edm', 'hiphop'), ('edm', 'rock')]>

Split 2
<accuracy=0.2926829268292683, num_incorrect=29, num_correct=12, errors=[('edm', 'country'), ('edm', 'rock'), ('edm', 'rock'), ('edm', 'pop'), ('edm', 'pop'), ('edm', 'rock'), ('edm', 'rock'), ('hiphop', 

Although this accuracy is not ideal, it is higher than the expected ZeroR classification accuracy (in which we choose a single output label for all inputs). For example, the accuracy for ZeroR classification accuracy with EDM, we receive around 12%. We think that the reason for this low accuracy is the high amount of output labels and noise across attribute values. We believe that we classification would be much more accurate at a more granular level, such as a binary classification task between two genres rather than a classification task among 7 genres where there can exist several overlapping characteristics.